<a href="https://colab.research.google.com/github/ancestor9/2025_Winter_Deep-Learning-with-TensorFlow/blob/main/20260107_03_Image%20Classification/pytorch_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.packtpub.com/en-us/product/python-deep-learning-9781837638505

We'll start by selecting device (GPU with fallback on CPU):

In [1]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Load the training dataset `train_data` with data augmentation `train_transform` and batch loader `train_loader`:

In [2]:
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader

# Training dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
])

train_data = datasets.CIFAR10(
    root='data',
    train=True,
    download=True,
    transform=train_transform)

batch_size = 50
train_loader = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2)

100%|██████████| 170M/170M [00:03<00:00, 44.8MB/s]


Load the validation dataset (no augmentations for validation):

In [3]:
validation_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225])
])

validation_data = datasets.CIFAR10(
    root='data',
    train=False,
    download=True,
    transform=validation_transform)

validation_loader = DataLoader(
    dataset=validation_data,
    batch_size=100,
    shuffle=True)

Define the CNN model:

In [4]:
torch.manual_seed(1234)
classes = 10

from torch.nn import Sequential, Conv2d, BatchNorm2d, GELU, MaxPool2d, Dropout2d, Linear, Flatten

model = Sequential(
    Conv2d(in_channels=3, out_channels=32,
           kernel_size=3, padding=1),
    BatchNorm2d(32),
    GELU(),
    Conv2d(in_channels=32, out_channels=32,
           kernel_size=3, padding=1),
    BatchNorm2d(32),
    GELU(),
    MaxPool2d(kernel_size=2, stride=2),
    Dropout2d(0.2),

    Conv2d(in_channels=32, out_channels=64,
           kernel_size=3, padding=1),
    BatchNorm2d(64),
    GELU(),
    Conv2d(in_channels=64, out_channels=64,
           kernel_size=3, padding=1),
    BatchNorm2d(64),
    GELU(),
    MaxPool2d(kernel_size=2, stride=2),
    Dropout2d(p=0.3),

    Conv2d(in_channels=64, out_channels=128,
           kernel_size=3),
    BatchNorm2d(128),
    GELU(),
    Conv2d(in_channels=128, out_channels=128,
           kernel_size=3),
    BatchNorm2d(128),
    GELU(),
    MaxPool2d(kernel_size=2, stride=2),
    Dropout2d(p=0.5),
    Flatten(),

    Linear(512, classes),
)

Define the `train_model` function, which implements the training:

In [5]:
def train_model(model, cost_function, optimizer, data_loader):
    # send the model to the GPU
    model.to(device)

    # set model to training mode
    model.train()

    current_loss = 0.0
    current_acc = 0

    # iterate over the training data
    for i, (inputs, labels) in enumerate(data_loader):
        # send the input/labels to the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            # forward
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = cost_function(outputs, labels)

            # backward
            loss.backward()
            optimizer.step()

        # statistics
        current_loss += loss.item() * inputs.size(0)
        current_acc += torch.sum(predictions == labels.data)

    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc.double() / len(data_loader.dataset)

    print('Train Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))

Define the `test_model` function, which implements the testing:

In [6]:
def test_model(model, cost_function, data_loader):
    # send the model to the GPU
    model.to(device)

    # set model in evaluation mode
    model.eval()

    current_loss = 0.0
    current_acc = 0

    # iterate over  the validation data
    for i, (inputs, labels) in enumerate(data_loader):
        # send the input/labels to the GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = cost_function(outputs, labels)

        # statistics
        current_loss += loss.item() * inputs.size(0)
        current_acc += torch.sum(predictions == labels.data)

    total_loss = current_loss / len(data_loader.dataset)
    total_acc = current_acc.double() / len(data_loader.dataset)

    print('Test Loss: {:.4f}; Accuracy: {:.4f}'.format(total_loss, total_acc))

    return total_loss, total_acc

Run the training and the validation for 50 epochs:

In [ ]:
cost_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

epochs = 50
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch + 1, epochs))
    train_model(model, cost_func, optimizer, train_loader)
    test_model(model, cost_func, validation_loader)

Epoch 1/50
Train Loss: 1.6602; Accuracy: 0.3868
Test Loss: 1.3239; Accuracy: 0.5134
Epoch 2/50
Train Loss: 1.3546; Accuracy: 0.5065
Test Loss: 1.1069; Accuracy: 0.6020
Epoch 3/50
Train Loss: 1.2057; Accuracy: 0.5670
Test Loss: 0.9729; Accuracy: 0.6523
Epoch 4/50
Train Loss: 1.1136; Accuracy: 0.6040
Test Loss: 0.9101; Accuracy: 0.6736
Epoch 5/50
Train Loss: 1.0380; Accuracy: 0.6303
Test Loss: 0.8619; Accuracy: 0.6938
Epoch 6/50
Train Loss: 0.9846; Accuracy: 0.6517
Test Loss: 0.8176; Accuracy: 0.7049
Epoch 7/50
Train Loss: 0.9367; Accuracy: 0.6704
Test Loss: 0.7619; Accuracy: 0.7264
Epoch 8/50
Train Loss: 0.8965; Accuracy: 0.6829
Test Loss: 0.7231; Accuracy: 0.7444
Epoch 9/50
Train Loss: 0.8622; Accuracy: 0.6976
Test Loss: 0.7238; Accuracy: 0.7472
Epoch 10/50
Train Loss: 0.8373; Accuracy: 0.7050
Test Loss: 0.6973; Accuracy: 0.7524
Epoch 11/50
